# Test Driven Model Development Using a Nice Sphere
Tim Tyree<br>
6.26.2020

In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  

#automate the boring stuff
from IPython import utils
import time, os, sys, re
beep = lambda x: os.system("echo -n '\\a';sleep 0.2;" * x)
if not 'nb_dir' in globals():
    nb_dir = os.getcwd()
sys.path.append("../lib") 
from lib import *
sys.path.append("lib") 
from lib import *

# from operari import *
# from ProgressBar import *
# from mesh_ops import *

# the visualization tools involved here for triangular meshes is
import trimesh
import pyglet
from numba import njit, cuda
# from numba.typed import List
# import numba
import trimesh

#try using a scipy sparse matrix to speed up spring force evaluations
#TODO: speed up bigger meshes with pycuda's sparce matrices
from scipy.sparse import csr_matrix
import scipy.sparse as sparse

#formating
from matplotlib import rc
# rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
# import matplotlib as mpl
# mpl.rcParams['font.size'] = 20
# mpl.rcParams['text.usetex'] = True
# mpl.rcParams['text.latex.preamble'] = r'\usepackage{{amsmath}}'
rc('text', usetex=True)

%autocall 1
%load_ext autoreload
%autoreload 2

Automatic calling is: Smart


# import the spherical mesh

In [2]:
os.chdir(f'{nb_dir}/Data/spherical_meshes')
mesh = trimesh.load('spherical_mesh_64.stl')
#subtract the center of mass
mesh.vertices -= mesh.center_mass
#normalize the mean radius to R
# R = 1. #unit length
# mesh.vertices *= R/np.mean(np.linalg.norm(mesh.vertices,axis=1))
mesh.vertices /= np.cbrt(mesh.volume*3/(4*np.pi))

face_normals all zero, ignoring!


In [3]:
assert(mesh.is_watertight)
assert(mesh.is_winding_consistent)

In [4]:
#unit length
print(np.mean(np.linalg.norm(mesh.vertices,axis=1)))
print(np.cbrt(mesh.volume*3/(4*np.pi)))

1.0083893459591489
0.9999999999999999


In [5]:
#doesn't work. don't know why
# def mat_to_str(A):
#     string = f'''
#     {A[[0,0]]} & {A[[0,1]]} & {A[[0,2]]} & {A[[0,3]]} \\ 
#     {A[[1,0]]} & {A[[1,1]]} & {A[[1,2]]} & {A[[1,3]]} \\ 
#     {A[[2,0]]} & {A[[2,1]]} & {A[[2,2]]} & {A[[2,3]]} \\ 
#     {A[[3,0]]} & {A[[3,1]]} & {A[[3,2]]} & {A[[3,3]]} \\ 
#     '''
#     latex_string = r'$\left( \begin{array}{ll} ' + string + r'\end{{array}} \right)$'
#     return latex_string
# # text(my_matrix, (1,1))

# [deprecated, use real space] cast all vertices and shapes to quaternions in numpy. record as the undeformed surface
- TODO: dilate the sphere by a known amount
- TODO: compute the deformation gradient
- TODO: compute the 1st piola-kirchoff tensor for some constitutive relation
- TODO: taking thickness to be $\delta\ll1$, compute the effective surface tension, $\gamma$

In [6]:
from colorsys import hls_to_rgb

def colorize(z):
    n,m = z.shape
    c = np.zeros((n,m,3))
    c[np.isinf(z)] = (1.0, 1.0, 1.0)
    c[np.isnan(z)] = (0.5, 0.5, 0.5)

    idx = ~(np.isinf(z) + np.isnan(z))
    A = (np.angle(z[idx]) + np.pi) / (2*np.pi)
    A = (A + 0.5) % 1.0
    B = 1.0 - 1.0/(1.0+abs(z[idx])**0.3)
    c[idx] = [hls_to_rgb(a, b, 0.8) for a,b in zip(A,B)]
    return c

In [7]:
#define the constant pauli matrices as a basis for real space
sigx = 1.j*np.array([[0.,1.],[1.,0.]],dtype=complex)
sigy = 1.j*np.array([[0.,-1.j],[1.j,0.]],dtype=complex)
sigz = 1.j*np.array([[1.,0.],[0.,-1.]],dtype=complex)
one  = np.array([[1.,0.],[0.,1.]],dtype=complex)
zero = np.array([[0.,0.],[0.,0.]],dtype=complex)

#print the imaginary spatial basis, and test that identity and zero multiply correctly.
print(r'$\sigma_x is $')
print(sigx)
print(r'$\sigma_y is $')
print(sigy)
print(r'$\sigma_z is $')
print(sigz)
# print(one)
# print(zero)
# print((one.dot(sigx)))
assert((one.dot(sigx)==sigx).all())
assert((zero.dot(sigx)==zero).all())

# @njit
def vec_to_quaternion(v):
    #define the constant pauli matrices as a basis for real space
    #     sigx = np.array([[0.,1.],[1.,0.]],dtype=complex)
    #     sigy = np.array([[1.j,0.],[0.,-1.j]],dtype=complex)
    #     sigz = np.array([[1.,0.],[0.,-1.]],dtype=complex)
    return v[0]*sigx + v[1]*sigy + v[2]*sigz

$\sigma_x is $
[[0.+0.j 0.+1.j]
 [0.+1.j 0.+0.j]]
$\sigma_y is $
[[ 0.+0.j  1.-0.j]
 [-1.+0.j  0.+0.j]]
$\sigma_z is $
[[ 0.+1.j  0.+0.j]
 [ 0.+0.j -0.-1.j]]


In [8]:
#test the inverse of the basis elements are minus the basis elements
assert((np.linalg.inv(sigx)==-sigx).all())
assert((np.linalg.inv(sigy)==-sigy).all())
assert((np.linalg.inv(sigz)==-sigz).all())
#test the basis elements are anticommuting
assert((sigx*sigy-sigy*sigx==zero).all())
assert((sigx*sigz-sigz*sigx==zero).all())
assert((sigz*sigy-sigy*sigz==zero).all())


In [9]:
trim = mesh.triangles[0]
tris = mesh.triangles[100]

dm1 = trim[1]-trim[0]
dm2 = trim[2]-trim[0]
Am =  np.cross(dm1,dm2)/2

ds1 = tris[1]-tris[0]
ds2 = tris[2]-tris[0]
As =  np.cross(ds1,ds2)/2


qm1 = dm1[0]*sigx + dm1[1]*sigy + dm1[2]*sigz
qm2 = dm2[0]*sigx + dm2[1]*sigy + dm2[2]*sigz
qs1 = ds1[0]*sigx + ds1[1]*sigy + ds1[2]*sigz
qs2 = ds2[0]*sigx + ds2[1]*sigy + ds2[2]*sigz

qAm =  Am[0]*sigx + Am[1]*sigy + Am[2]*sigz
qAs =  As[0]*sigx + As[1]*sigy + As[2]*sigz

# solve for the deformation gradient between either pair of edges.
F1 = qs1.dot(np.linalg.inv(qm1))
F2 = qs2.dot(np.linalg.inv(qm2))
# FA = qAs.dot(np.linalg.inv(qAm))

In [10]:
#singular value decomposition on the block diagonal deformations
U1, S1, V1 = np.linalg.svd(F1)
V1 = V1.T.conjugate()

U2, S2, V2 = np.linalg.svd(F2)
V2 = V2.T.conjugate()

In [11]:
#verify svd is returning hermitian factors
assert((np.around(U1.dot(U1.T.conjugate()),3)==one).all())
assert((np.around(V1.dot(V1.T.conjugate()),3)==one).all())

In [12]:
Dm = np.array(((qm1,zero),(zero,qm2))).reshape((4,4))
Ds = np.array(((qs1,zero),(zero,qs2))).reshape((4,4))

In [13]:
#TODO: test if rotating by F maps dm to ds
F1.dot(qm1).dot(F1.conjugate().T)

array([[ 0.00000000e+00-0.0020773j , -2.64751533e-01-0.23108376j],
       [ 2.64751533e-01-0.23108376j, -6.93889390e-18+0.0020773j ]])

In [14]:
np.linalg.svd(Dm)

(array([[-8.89379716e-14+8.48789279e-01j,  1.11022302e-16+5.28731274e-01j,
         -2.64271567e-17-3.88827512e-17j, -2.62956193e-33+3.33435694e-32j],
        [-1.11022302e-16-7.60839488e-17j,  2.16840434e-19+1.79906187e-19j,
         -8.74768222e-01-4.84541596e-01j,  6.23283209e-17+9.18756336e-17j],
        [ 0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         -2.22044605e-16-1.62091656e-17j, -9.97589537e-01-6.93910349e-02j],
        [-5.58442181e-14+5.28731274e-01j,  1.37368415e-16-8.48789279e-01j,
         -1.65169697e-17-2.44628975e-17j, -1.64347588e-33+2.08773777e-32j]]),
 array([3.77416525e-01, 3.00009770e-01, 1.64880887e-18, 5.41184667e-36]),
 array([[ 5.84178464e-04+0.00000000e+00j,  5.38170272e-01+4.58663730e-01j,
          5.38170272e-01-4.58663730e-01j, -5.84178464e-04-5.55111512e-17j],
        [ 4.91868520e-01+0.00000000e+00j,  3.29205021e-01-3.86897161e-01j,
          3.29205021e-01+3.86897161e-01j, -4.91868520e-01-2.40692882e-17j],
        [ 1.223286

In [15]:
#TODO: visualize F1 and F2 and see if they make sense.

TODO: matrix to quaternion

TODO: quaternion to matrix

In [16]:
sigy.T.conjugate()

array([[ 0.-0.j, -1.-0.j],
       [ 1.+0.j,  0.-0.j]])

In [17]:
sigy

array([[ 0.+0.j,  1.-0.j],
       [-1.+0.j,  0.+0.j]])

# Handling Stress with quaternions only with nearly conformal mappings

Two deformations are conformal if their differentials are related by a rotation and a scaling.

$$d\tilde f = e^uRdf$$

Rotations represented in $\mathbb{R}^3$ require quadratic constraints which produce shear stress, which causes angles change under deformation.  The action of a quaternion on a vector can be written in terms of a rotation and a scale.

The shape of a triangle, which has six degrees of freedom in $\mathbb{R}^3$ cannot be represented by a single quaternion, which has only four degrees of freedom.  It is therefore necessary to represent the shape of a triangle with at least two quaternions, and it is sufficient to constrain those quaternions to be imaginary, since Im$\,\mathbb{H}\simeq\mathbb{R}^3$.

# Enter the basis of the triangle in real space only to compute deformation gradients that support shear stress

In [18]:
#DONE: copy the 'rotate into same plane solution' to a .py file
#DONE: rotate two triangles into the same plane


In [21]:
tris = mesh.triangles[100]
trim = mesh.triangles[321]

In [24]:
dm1 = trim[1]-trim[0]     #precompute in final algorithm
dm2 = trim[2]-trim[0]     #precompute in final algorithm
Am =  np.cross(dm1,dm2)/2 #precompute in final algorithm

ds1 = tris[1]-tris[0]
ds2 = tris[2]-tris[0]
As =  np.cross(ds1,ds2)/2

#Ra is a rotation_matrix that rotation_matrix rotates Ashat onto Amhat
Ashat = As/np.linalg.norm(As)
Amhat = Am/np.linalg.norm(Am)

In [26]:
R = get_R(trim,tris)

In [27]:
R

array([[-0.79946888, -0.25815147,  0.54240883],
       [-0.4627009 ,  0.84047528, -0.28197373],
       [ 0.38308928,  0.47640228,  0.79138074]])

In [29]:
np.linalg.norm(R.dot(dm1))/np.linalg.norm(dm1)

1.0000000000000002

In [36]:
#test that the local "x" axis is aligned by R
xhat = ds1/np.linalg.norm(ds1)
assert(np.isclose(xhat,R.dot(dm1)/np.linalg.norm(dm1)).all())

In [39]:
#yhat is not needed for energy calculation, but is needed for deformation gradient calculation via outer product
yhat = np.cross(As,xhat)
yhat /= np.linalg.norm(yhat)

In [ ]:
#TODO: project all  edges onto the first vector using the outer product.  Test that the first vectors match. 
#TODO: for each second vector, compute the orthogonal component via the pythagorean theorem
#TODO: with 2 dimensional vectors in hand, plot some vectors on top of eachother in two different colors.
#TODO: use a shear deformation from the heisenburg group to make the triangles match
#TODO: test ^this by ploting a number of triangles.  do they all match?
#TODO: if so, congrats!  Collect the operations into one matrix.  congrats!  you can now measure the deformation gradient F!

In [ ]:
#TODO: scale so "the first" edge matches between the two triangles
